import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=900
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Worms0(i)

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [33 13 10 13  8]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.2

1 33	1 33	2 13	3 10	4 13	

In [3]:
np.shape(ecg)

(102, 901)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 33	2 13	3 10	4 13	5 8	
 max = 33


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (33, 901)
Up to class  2 train shape =  (46, 901)
Up to class  3 train shape =  (56, 901)
Up to class  4 train shape =  (69, 901)
Up to class  5 train shape =  (77, 901)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 33 13 10 13  8]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 5., 5., 5., 5., 5., 5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.1375
min magnitude class 1  =  -4.037800000000001
after normalizing >>
max magnitude class 1  =  0.9870973424048264
min magnitude class 1  =  -0.8129555012440277

 cls 2  >> 
max magnitude class 2  =  4.8591000000000015
min magnitude class 2  =  -4.3114
after normalizing >>
max magnitude class 2  =  1.1459805799590468
min magnitude class 2  =  -0.8731972609375345

 cls 3  >> 
max magnitude class 3  =  4.263200000000001
min magnitude class 3  =  -3.2160000000000006
after normalizing >>
max magnitude class 3  =  1.014774203492085
min magnitude class 3  =  -0.6320100402932824

 cls 4  >> 
max magnitude class 4  =  2.804300000000001
min magnitude class 4  =  -3.1806000000000005
after normalizing >>
max magnitude class 4  =  0.6935508730211157
min magnitude class 4  =  -0.6242156020873243

 cls 5  >> 
max magnitude class 5  =  2.7693
min magnitude class 5  =  -3.8073000000000006
after normalizing >>
max magnitude class 5  =  0.6858445075632471
min ma

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.8731972609375345
max tst = 1.1459805799590468


In [14]:
len(wndws_test)

181

In [15]:
np.shape(wndws_test)

(181, 901)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(33, 901)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (77, 900)
y_train =>  (77,)
X_test  =>  (181, 900)
y_test  =>  (181,)
X_valid  =>  (77, 900)
y_valid  =>  (77,)


y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [30]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 70
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data
input_shape=np.shape(X_train)
output_shape=np.shape(y_train_cat)[1]
input = Input(shape=(input_shape[1], input_shape[2]))

'''block_1'''
b1_cnv1d_1 = Conv1D(filters=32, kernel_size=(3), strides=1, padding='same',
                 use_bias=False, name='b1_cnv1d_1', kernel_initializer='normal')(input)
b1_relu_1 = ReLU(name='b1_relu_1')(b1_cnv1d_1)
b1_bn_1 = b1_relu_1#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b1_bn_1')(b1_relu_1)  # size: 14*14
b1_pul=MaxPool1D(pool_size=3,strides=3)(b1_bn_1)

b1_cnv1d_2 = Conv1D(filters=64, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b1_cnv1d_2', kernel_initializer='normal')(b1_pul)
b1_relu_2 = ReLU(name='b1_relu_2')(b1_cnv1d_2)
b1_out = b1_relu_2#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b1_bn_2_out')(b1_relu_2)  # size: 14*14

'''block 2'''
b2_cnv1d_1 = Conv1D(filters=64, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b2_cnv1d_1', kernel_initializer='normal')(b1_out)
b2_relu_1 = ReLU(name='b2_relu_1')(b2_cnv1d_1)
b2_bn_1 = b2_relu_1#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b2_bn_1')(b2_relu_1)  # size: 14*14

b2_add = add([b1_out, b2_bn_1], name='b2_add')  #

b2_pul=MaxPool1D(pool_size=3,strides=3)(b2_add)

b2_cnv1d_2 = Conv1D(filters=128, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b2_cnv1d_2', kernel_initializer='normal')(b2_pul)
b2_relu_2 = ReLU(name='b2_relu_2')(b2_cnv1d_2)
b2_out = b2_relu_2#BatchNormalization(epsilon=1e-3, momentum=0.999,name='b2_out')(b2_relu_2)  # size: 7*7



'''block 3'''
b3_cnv1d_1 = Conv1D(filters=128, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b3_cnv1d_1', kernel_initializer='normal')(b2_out)
b3_relu_1 = ReLU(name='b3_relu_1')(b3_cnv1d_1)
b3_bn_1 = b3_relu_1#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b3_bn_1')(b3_relu_1)  # size: 7*7

b3_add = add([b2_out, b3_bn_1], name='b3_add')  #
b3_pul=MaxPool1D(pool_size=3,strides=1, name='p3_pul')(b3_add)

b3_cnv1d_2 = Conv1D(filters=256, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b3_cnv1d_2', kernel_initializer='normal')(b3_pul)
b3_relu_2 = ReLU(name='b3_relu_2')(b3_cnv1d_2)
b3_bn_2 = b3_relu_2#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b3_bn_2')(b3_relu_2)  # size: 3*3

b3_cnv1d_3 = Conv1D(filters=256, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b3_cnv1d_3', kernel_initializer='normal')(b3_bn_2)
b3_relu_3 = ReLU(name='b3_relu_3')(b3_cnv1d_3)
b3_out = b3_relu_3#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b3_out')(b3_relu_3)  # size: 3*3

'''block 4'''
b4_cnv1d_1 = Conv1D(filters=256, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b4_cnv1d_1', kernel_initializer='normal')(b3_out)
b4_relu_1 = ReLU(name='b4_relu_1')(b4_cnv1d_1)
b4_bn_1 = b4_relu_1#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b4_bn_1')(b4_relu_1)  # size: 7*7

b4_add = add([b3_out, b4_bn_1], name='b4_add')  #
b4_pul=MaxPool1D(pool_size=2,strides=1, name='b4_pul')(b4_add)

b4_cnv1d_2 = Conv1D(filters=512, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b4_cnv1d_2', kernel_initializer='normal')(b4_add)
b4_relu_2 = ReLU(name='b4_relu_2')(b4_cnv1d_2)
b4_bn2 = b4_relu_2#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b4_bn_2')(b4_relu_2)  # size: 3*3

b4_cnv1d_3 = Conv1D(filters=512, kernel_size=3, strides=1, padding='same',
                    use_bias=False, name='b4_cnv1d_3', kernel_initializer='normal')(b4_bn2)
b4_relu_3 = ReLU(name='b4_relu_3')(b4_cnv1d_3)
b4_out = b4_relu_3#BatchNormalization(epsilon=1e-3, momentum=0.999, name='b4_bn_3_out')(b4_relu_3)  # size: 3*3

'''block 5'''
b5_avg_p =Flatten()(b4_out)
#b5_avg_p = GlobalAveragePooling1D()(b4_out)

output1 = Dense(4096, name='model_output1', activation='softmax',
               kernel_initializer='he_uniform')(b5_avg_p)
                                                
output2 = Dense(4096, name='model_output2', activation='softmax',
               kernel_initializer='he_uniform')(output1)
                                                
output3 = Dense(output_shape, name='model_output3', activation='softmax',
               kernel_initializer='he_uniform')(output2)

model_Res = Model(input, output3)

model_Res.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
history = model_Res.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
'''_, accuracy = model_Res.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)
print('accuracy = ', accuracy)
y_pred_Res = model_Res(X_test)
#model_Res.save('P0103CNN_fld1_win03_Residual_2.h5')'''

Epoch 1/1000


ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model_2/model_output1/MatMul/MatMul_1' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 596, in run_fo

In [ ]:
model_Res.summary()

predict_x=model_Res.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)